In [1]:
import pandas as pd
import ktrain
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import pickle

/home/j/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
xgb = XGBClassifier()
xgb.load_model('/home/j/Documents/Projects/social-media-combat-detection/models/xgb_classifier')
with open('/home/j/Documents/Projects/social-media-combat-detection/models/tfid-vectorizer.pickle', 'rb') as handle:
        vectorizer = pickle.load(handle)

In [3]:
df = pd.read_csv('/home/j/Documents/Projects/social-media-combat-detection/data/test.csv')

df = df[(df['annotation'] == '1') | (df['annotation'] == '0')]

df.head()

,idx,msg_txt,annotation
0,2983,DIU Russians modernized X-22 and Oniks missi...,0
1,1612,Wagner currently controls 4.8 thousand km2 o...,0
2,35,At the positions of the artillery of the 32n...,0
4,2332,We receive reports that there are power outa...,0
5,330,The law on electronic subpoenas equates Russ...,0


In [4]:
freq_matrix_test = vectorizer.transform(df['msg_txt'])

In [5]:
y_pred = xgb.predict(freq_matrix_test)

print(classification_report(df['annotation'].astype(int),y_pred))

              precision    recall  f1-score   support

           0       0.89      0.96      0.93       376
           1       0.85      0.66      0.74       127

    accuracy                           0.88       503
   macro avg       0.87      0.81      0.83       503
weighted avg       0.88      0.88      0.88       503



In [15]:
predictor = ktrain.load_predictor('/home/j/Documents/Projects/social-media-combat-detection/models/bert')

/home/j/.local/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


In [16]:
predictions = predictor.predict(df['msg_txt'].to_list())

In [17]:
pred_ints = []
for x in predictions:
    if x == 'annotation':
        pred_ints.append(1)
    else:
        pred_ints.append(0)

In [18]:
print(classification_report(df['annotation'].astype(int),pred_ints))

              precision    recall  f1-score   support

           0       0.97      0.87      0.92       376
           1       0.70      0.93      0.80       127

    accuracy                           0.88       503
   macro avg       0.84      0.90      0.86       503
weighted avg       0.90      0.88      0.89       503

